# References

- <https://github.com/tylere/PyDataNYC2017/blob/master/ipynb/satellite_analysis.ipynb>
- <https://github.com/tylere/ee-jupyter-examples/blob/master/2%20-%20EE%20101.ipynb>
- <https://ipyleaflet.readthedocs.io/en/latest/api_reference/map.html>
- <https://github.com/tylere/ee-jupyter-examples/blob/master/4%20-%20Earth%20Engine%20and%20TensorFlow.ipynb>
- <https://developers.google.com/earth-engine/api_docs>

In [2]:
from main import ee, feats
print(feats.keys())

dict_keys(['Tailing', 'Processing_Facilities', 'Dumps', 'Active_Mines', 'Artisinal_Mining', 'Smelters', 'Exploration'])


In [88]:
import ipyleaflet
from ipywidgets import HTML

# Setup interactive map
ee_map = ipyleaflet.Map(zoom=3, layout={'height':'400px'}, scroll_wheel_zoom=True)

def GetTileLayerUrl(ee_image_object):
    """Load tile url from EE for a given image object"""
    map_id = ee.Image(ee_image_object).getMapId()
    tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
    return tile_url_template.format(**map_id)

def ResetLayers(map_object):
    """Clear map overlays"""
    for lyr in map_object.layers[1:]:
        map_object.remove_layer(lyr)

def AddTileLayer(map_object, ee_image_object):
    """Add EE tile overlay"""
    tile_url = GetTileLayerUrl(ee_image_object)
    map_object.add_layer(ipyleaflet.TileLayer(url=tile_url))

def AddPointsLayer(map_object, feats):
    """Add point features overlay"""
    geojson = feats.getInfo()
    markers = []
    for feat in geojson['features']:
        geo = feat['geometry']
        if geo['type'] != 'Point': continue
        msg = HTML()
        msg.value = feat['properties']['name']
        msg.description = feat['properties']['description']
        lng, lat = geo['coordinates']
        marker = ipyleaflet.Marker(location=(lat, lng))
        marker.popup = msg
        markers.append(marker)

    marker_cluster = ipyleaflet.MarkerCluster(
        markers=markers
    )
    map_object.add_layer(marker_cluster)

In [3]:
# "Landsat 8 image collection
# which has been processed to surface reflectance values
# (i.e. the processing has attempted to remove the atmospheric effects)."
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

sample_image = ee.Image(
    l8sr.filterDate('2017-11-01', '2017-12-01')
        .filterBounds(ee.Geometry.Point(-73.9957, 40.7262))
        .first()
)
band_names_original = sample_image.bandNames()

l8_bands = ee.Dictionary({
    'B1': 'ultra_blue',
    'B2': 'blue',
    'B3': 'green',
    'B4': 'red',
    'B5': 'nir',
    'B6': 'swir_1',
    'B7': 'swir_2',
    'B8': 'pan',
    'B9': 'cirrus',
    'B10': 'tirs_1',
    'B11': 'tirs_2',
    'sr_aerosol': 'sr_aerosol', 
    'pixel_qa': 'pixel_qa',
    'radsat_qa': 'radsat_qa'
})
band_names_new = l8_bands.values(sample_image.bandNames())
l8sr = l8sr.select(band_names_original, band_names_new)

In [89]:
ResetLayers(ee_map)

In [90]:
filtered = (
    l8sr.filterDate('2017-01-01', '2017-01-02')
        .median()
#         .mean()
#         .max()
#         .min()
#         .reduce(ee.Reducer.percentile([25])).rename(band_names_new)
        .visualize(min=0, max=3000, bands=['red', 'green', 'blue'])
)
ReplaceOverlayLayers(ee_map, filtered)

In [91]:
AddPointsLayer(ee_map, feats['Active_Mines'])

In [92]:
ee_map

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …